In [153]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

from scipy.integrate import simpson


from scipy import stats

from multiprocessing import Pool, cpu_count
from conf_interval import conf_interval_calc



%load_ext autoreload
%autoreload 2

from pairedtest import pairedtest
# regex for number extraction from string
number_pattern =  r'(-?(?:0|[1-9]\d*)(?:\.\d+)?(?:[eE][+-]?\d+)?)'

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':200})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

disjoint_set_size_threshold = 10

In [56]:
tf_df_list = [
    pd.read_csv('../output/network_model/yeast_tf.csv', index_col=0, names=['tf']),
    pd.read_csv('../output/network_model/bsubtilis_tf.csv', index_col=0, names=['tf']),
    pd.read_csv('../output/network_model/arabidopsis_tf.csv', index_col=0, names=['tf']),
    pd.read_csv('../output/network_model/mouse_tf.csv', index_col=0, names=['tf']),
]

res_df_list = [
    pd.read_csv('../output/network_model/yeast_all_tf_high_var_target_efron_train.csv.gz', index_col=0, compression='gzip'),
    pd.read_csv('../output/network_model/bsubtilis_all_tf_high_var_target_efron_train.csv.gz', index_col=0, compression='gzip'),
    pd.read_csv('../output/network_model/arabidopsis_all_tf_high_var_target_efron_train.csv.gz', index_col=0, compression='gzip'),
    pd.read_csv('../output/network_model/mouse_all_tf_high_var_target_efron_train.csv.gz', index_col=0, compression='gzip'),
]
species_file_names = ['yeast', 'bsubtilis', 'arabidopsis', 'mouse']

In [59]:
for res_df, tf_list, species_file_name in zip(res_df_list, tf_df_list, species_file_names):
    out_df = res_df[['rf_rmse', 'test_std', 'rf_efron_features', 'rf_efron_complementary_features_list']]
    disjoint_sets_list = []
    minimal_sets_list = []
    for i, row in out_df.iterrows():
        first_set = row['rf_efron_features'].split('; ')
        first_set = ': '.join([tf_list.index[int(j)] for j in first_set])
        minimal_sets_list.append(first_set)
        disjoint_sets = [first_set]
        if isinstance(row['rf_efron_complementary_features_list'], str):
            rest_of_sets = row['rf_efron_complementary_features_list'].split(', ')
            for disjoint_set in rest_of_sets:
                disjoint_sets.append(': '.join([tf_list.index[int(j)] for j in disjoint_set.split('; ')]))
        # print(disjoint_sets)
        # break
        disjoint_sets_list.append('; '.join(disjoint_sets))
    out_df['disjoint_sets'] = disjoint_sets_list
    out_df['minimal_set'] = minimal_sets_list
    out_df.index.name = 'target_gene'
    out_df[['minimal_set', 'disjoint_sets']].to_csv('../output/network_model/{}_disjoint_sets.csv'.format(species_file_name))

/tmp/ipykernel_2152906/491479083.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_df['disjoint_sets'] = disjoint_sets_list
/tmp/ipykernel_2152906/491479083.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_df['minimal_set'] = minimal_sets_list
/tmp/ipykernel_2152906/491479083.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [309]:
res_df = pd.read_csv('../output/network_model/arabidopsis_all_tf_high_var_target_efron_train.csv.gz', index_col=0, compression='gzip')
tf_list = pd.read_csv('../output/network_model/arabidopsis_tf.csv', index_col=0, names=['tf'])
out_df = res_df[['rf_rmse', 'test_std', 'rf_efron_features', 'rf_efron_complementary_features_list']]


In [227]:
pathway_genes = pd.read_csv('../data/arabidopsis_nitro_pathway.tsv.gz', sep='\t', compression='gzip')

In [228]:
gene_names = pathway_genes[pathway_genes['Gene Names'].notna()]['Gene Names'].values
gene_names = [s.split(' ') for s in gene_names]
gene_names = [
    x.upper()
    for xs in gene_names
    for x in xs
]

In [229]:
pathway_matches = []
for ind in out_df.index:
    if ind in gene_names:
        pathway_matches.append(ind)

In [230]:
out_df = out_df.loc[pathway_matches]

In [185]:
tair_genes = pd.read_csv('../data/tair_nitrogen.csv', sep='\t')
tair_genes['Locus']

0      AT3G26680
1      AT3G26680
2      AT5G65720
3      AT1G02860
4      AT1G53310
         ...    
101    AT1G10010
102    AT5G50950
103    AT4G24670
104    AT5G57390
105    AT3G03910
Name: Locus, Length: 106, dtype: object

In [189]:
pathway_matches = []
for ind in out_df.index:
    if ind in tair_genes['Locus'].values:
        pathway_matches.append(ind)

In [293]:
out_df = out_df.loc[pathway_matches]

In [310]:
disjoint_sets_list = []
disjoint_sets_dict = {}
minimal_sets_list = []
for i, row in out_df.iterrows():
    first_set = row['rf_efron_features'].split('; ')
    first_set = set([int(i) for i in first_set])
    # first_set = ': '.join([tf_list.index[int(j)] for j in first_set])
    minimal_sets_list.append(first_set)
    if (len(first_set) > disjoint_set_size_threshold):
        disjoint_sets = []
    else:
        disjoint_sets = [first_set]
    if isinstance(row['rf_efron_complementary_features_list'], str):
        rest_of_sets = row['rf_efron_complementary_features_list'].split(', ')
        for n_set in rest_of_sets:
            disjoint_set = n_set.split('; ')
            disjoint_set = set([int(i) for i in disjoint_set])
            if (len(disjoint_set) <= disjoint_set_size_threshold):
                disjoint_sets.append(disjoint_set)
    # print(disjoint_sets)
    # break
    if (len(disjoint_sets) > 0):
        disjoint_sets_list.append(disjoint_sets)
        disjoint_sets_dict[i] = disjoint_sets

In [311]:
tf_occur_count = {}

In [312]:
for disjoint_sets in disjoint_sets_list:
    for disjoint_set in disjoint_sets:
        for tf in disjoint_set:
            if tf in tf_occur_count:
                tf_occur_count[tf] += 1
            else: tf_occur_count[tf] = 1

In [313]:
tf_occur_count = dict(sorted(tf_occur_count.items(), key=lambda item: item[1]))

In [314]:
tf_union_set = set(tf_occur_count.keys())
sorted_tf_list = [i for i in tf_occur_count.keys()]

In [315]:
continue_flag = True
tf_remove_index = 0
while (continue_flag):
    for disjoint_sets in disjoint_sets_list:
        should_continue = False
        for disjoint_set in disjoint_sets:
            if len(disjoint_set.intersection(tf_union_set)) == len(disjoint_set):
                should_continue = True
                break
        if not should_continue:
            continue_flag = False
            break
    if continue_flag:
        tf_union_set.remove(sorted_tf_list[tf_remove_index])
        tf_remove_index += 1

In [316]:
len(tf_occur_count)

2006

In [317]:
covered_targets = set()
remaining_targets = set(disjoint_sets_dict.keys())
used_tf = set()
tf_add_index = -1
while (len(remaining_targets)>0):
    used_tf.add(sorted_tf_list[tf_add_index])
    target_search_list = copy.deepcopy(list(remaining_targets))
    for target in target_search_list:
        for dj_set in disjoint_sets_dict[target]:
            if len(dj_set.intersection(used_tf)) == len(dj_set):
                remaining_targets.remove(target)
                covered_targets.add(target)
                break
    tf_add_index -= 1

In [318]:
used_tf_occur_count = {key: tf_occur_count[key] for key in used_tf}
used_tf_occur_count = dict(sorted(used_tf_occur_count.items(), key=lambda item: item[1]))

In [319]:
used_sorted_tf_list = [i for i in used_tf_occur_count.keys()]
used_tf_union_set = set(used_tf_occur_count.keys())


In [320]:
len(used_tf_union_set)

1931

In [321]:
len(used_sorted_tf_list)

1931

In [322]:
continue_flag = True
tf_remove_index = 0
while (tf_remove_index < len(used_sorted_tf_list)):
    continue_flag = True
    new_tf_set = copy.deepcopy(used_tf_union_set)
    new_tf_set.remove(used_sorted_tf_list[tf_remove_index])
    for disjoint_sets in disjoint_sets_list:
        should_continue = False
        for disjoint_set in disjoint_sets:
            if len(disjoint_set.intersection(new_tf_set)) == len(disjoint_set):
                should_continue = True
                break
        if not should_continue:
            continue_flag = False
            break
    if continue_flag:
        used_tf_union_set = new_tf_set
    tf_remove_index += 1
    

In [323]:
len(used_tf_union_set)

298

In [324]:
out_df.shape

(1397, 4)